# Task B — Modeling & Task C — Anomalies

Implement model and anomaly detection here.

In [1]:

import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from pathlib import Path
pd.set_option('display.max_columns', 120)
plt.style.use('seaborn-v0_8')
DATA_DIR = Path('../dataset')
sup = pd.read_csv(DATA_DIR/'suppliers.csv')
prod = pd.read_csv(DATA_DIR/'products.csv')
prices = pd.read_csv(DATA_DIR/'price_lists.csv', parse_dates=['valid_from','valid_to'])
po = pd.read_csv(DATA_DIR/'purchase_orders.csv', parse_dates=['order_date','promised_date'])
deliv = pd.read_csv(DATA_DIR/'deliveries.csv', parse_dates=['actual_delivery_date'])
po = po.merge(deliv, on='order_id', how='left')
print(sup.shape, prod.shape, prices.shape, po.shape)


(20, 7) (40, 5) (854, 7) (4307, 20)


In [2]:
# === Cell 2.1: Define target ===

# Ensure datetime types (already parsed on read, but re-assert just in case)
po["order_date"] = pd.to_datetime(po["order_date"], errors="coerce")
po["promised_date"] = pd.to_datetime(po["promised_date"], errors="coerce")
po["actual_delivery_date"] = pd.to_datetime(po["actual_delivery_date"], errors="coerce")

# Target: late_delivery = 1 if actual_delivery_date > promised_date
po["late_delivery"] = (po["actual_delivery_date"] > po["promised_date"]).astype(int)

print("Target distribution:", po["late_delivery"].value_counts(normalize=True))


Target distribution: late_delivery
0    0.502206
1    0.497794
Name: proportion, dtype: float64


### Target Definition  
We define the target variable **`late_delivery`** as 1 when the actual delivery date is later than the promised date, and 0 otherwise.  
This gives us a clear label for the prediction task and lets us quantify the share of late deliveries in the data.  


In [3]:
# === Cell 2.2: Currency normalization ===
USD_TO_EUR = 0.92

if "unit_price" in po.columns and "currency" in po.columns:
    po["unit_price_eur"] = po.apply(
        lambda x: x["unit_price"] * USD_TO_EUR if x["currency"] == "USD" else x["unit_price"], axis=1
    )
else:
    po["unit_price_eur"] = np.nan

print(po[["unit_price","currency","unit_price_eur"]].head())


   unit_price currency  unit_price_eur
0       11.81      EUR           11.81
1       22.43      EUR           22.43
2        7.50      EUR            7.50
3       10.65      EUR           10.65
4        8.11      EUR            8.11


### Currency Normalization  
Purchase orders are recorded in both EUR and USD.  
To make prices comparable across suppliers and products, we normalize everything to **EUR**, using the assumption from the exercise that **1 USD = 0.92 EUR**.  
The new column `unit_price_eur` will be used in modeling and analysis.  


In [5]:
# === Cell 2.3: Distance buckets ===
bins = [-1, 500, 1500, 3000, float("inf")]
labels = ["<500", "500-1499", "1500-2999", "3000+"]

if "distance_km" in po.columns:
    po["distance_bucket"] = pd.cut(po["distance_km"], bins=bins, labels=labels)
else:
    po["distance_bucket"] = "Unknown"

print(po[["distance_km","distance_bucket"]].head())


   distance_km distance_bucket
0          748        500-1499
1         1188        500-1499
2          857        500-1499
3          729        500-1499
4          205            <500


### Distance Buckets  
To better understand and model the effect of shipping distance, we group `distance_km` into four categories:  
- <500 km  
- 500–1499 km  
- 1500–2999 km  
- 3000+ km  

This feature can highlight patterns in performance across short- vs. long-haul deliveries.  



In [6]:
# === Cell 2.4 (revised): Select order-time features & robust imputation ===
from pandas.api.types import is_numeric_dtype, is_bool_dtype, is_categorical_dtype

feature_cols = [
    "supplier_rating", "preferred_supplier", "country", "ship_mode", "incoterm",
    "payment_terms", "hazardous_flag", "promised_lead_days", "urgent",
    "qty", "unit_price_eur", "distance_km", "distance_bucket"
]

use_cols = [c for c in feature_cols if c in po.columns]
df_model = po[use_cols + ["late_delivery","order_date"]].copy()

# Split columns by dtype in a robust way
numeric_or_bool = []
categorical_like = []
for c in use_cols:
    s = df_model[c]
    if is_numeric_dtype(s) or is_bool_dtype(s):
        numeric_or_bool.append(c)
    else:
        categorical_like.append(c)

# Impute numeric/bool with median (bools will be treated as 0/1 if missing occurs)
for c in numeric_or_bool:
    df_model[c] = pd.to_numeric(df_model[c], errors="coerce")  # ensure numeric
    df_model[c] = df_model[c].fillna(df_model[c].median())

# Impute categoricals safely (handle true pandas.Categorical and plain object)
for c in categorical_like:
    s = df_model[c]
    if is_categorical_dtype(s):
        # add "Unknown" to categories, then fill
        df_model[c] = s.cat.add_categories(["Unknown"]).fillna("Unknown")
    else:
        # make sure it's object/string, then fill
        df_model[c] = s.astype("object").fillna("Unknown")

# Clean target
df_model["late_delivery"] = df_model["late_delivery"].fillna(0).astype(int)

print("Prepared model dataset shape:", df_model.shape)
print("Numeric/bool cols:", numeric_or_bool)
print("Categorical-like cols:", categorical_like)


Prepared model dataset shape: (4307, 10)
Numeric/bool cols: ['urgent', 'qty', 'unit_price_eur', 'distance_km']
Categorical-like cols: ['ship_mode', 'incoterm', 'payment_terms', 'distance_bucket']


C:\Users\Rabbit User\AppData\Local\Temp\ipykernel_23216\2446119694.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(s):
C:\Users\Rabbit User\AppData\Local\Temp\ipykernel_23216\2446119694.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(s):
C:\Users\Rabbit User\AppData\Local\Temp\ipykernel_23216\2446119694.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(s):
C:\Users\Rabbit User\AppData\Local\Temp\ipykernel_23216\2446119694.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(

### Order-Time Features and Missing Values  
For modeling, we only keep information that is **available at the time of order** (e.g. supplier attributes, promised lead days, unit price, ship mode).  
We exclude any data that would only be known after delivery to avoid leakage.  

Missing values are handled as follows:  
- **Numeric fields** (e.g. lead days, unit price) → imputed with the median.  
- **Categorical fields** (e.g. ship mode, payment terms) → imputed with `"Unknown"`.  

This ensures the dataset is consistent and ready for model training.  


In [7]:
# === Cell 2.5: Temporal split ===
df_model["order_date"] = pd.to_datetime(df_model["order_date"], errors="coerce")

train_df = df_model[df_model["order_date"] <= "2025-03-31"].copy()
valid_df = df_model[(df_model["order_date"] >= "2025-04-01") & (df_model["order_date"] <= "2025-06-30")].copy()

X_train = train_df.drop(columns=["late_delivery","order_date"])
y_train = train_df["late_delivery"]
X_valid = valid_df.drop(columns=["late_delivery","order_date"])
y_valid = valid_df["late_delivery"]

print("Train:", X_train.shape, "Valid:", X_valid.shape)
print("Late rate — Train:", y_train.mean(), "| Valid:", y_valid.mean())


Train: (3572, 8) Valid: (735, 8)
Late rate — Train: 0.49384098544232924 | Valid: 0.5170068027210885


### Train/Validation Split  
We split the data on **order date** to mimic real-world prediction:  
- **Training set** → all orders up to 31 March 2025  
- **Validation set** → orders from 1 April to 30 June 2025  

This prevents future information from leaking into training and gives a realistic test of model performance on new data.  
We also check the late-delivery rate in each set to confirm that the split is representative.  


In [ ]:

df = po.query('cancelled == 0').copy()
df['late_delivery'] = df['late_delivery'].fillna(0).astype(int)
cutoff = pd.Timestamp('2025-03-31')
train = df[df['order_date'] <= cutoff].copy()
valid = df[df['order_date'] > cutoff].copy()

def engineer(d):
    out = d.copy()
    out['promised_lead_days'] = (out['promised_date'] - out['order_date']).dt.days
    out['month'] = out['order_date'].dt.month
    out = out.merge(sup[['supplier_id','preferred','rating']], on='supplier_id', how='left')
    out = out.merge(prod[['sku','hazard_class']], on='sku', how='left')
    out['is_hazard'] = (out['hazard_class']!='none').astype(int)
    out['is_eur'] = (out['currency']=='EUR').astype(int)
    out = pd.get_dummies(out, columns=['ship_mode','incoterm','payment_terms'], drop_first=True)
    return out

X_train = engineer(train); X_valid = engineer(valid)
y_train = X_train['late_delivery']; y_valid = X_valid['late_delivery']
cols_drop = ['order_id','order_date','promised_date','actual_delivery_date','order_notes','sku','currency','hazard_class','late_delivery']
X_train = X_train.drop(columns=cols_drop, errors='ignore')
X_valid = X_valid.drop(columns=cols_drop, errors='ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score
clf = RandomForestClassifier(n_estimators=300, random_state=0, class_weight='balanced')
clf.fit(X_train, y_train)
p_valid = clf.predict_proba(X_valid)[:,1]
print('PR-AUC:', average_precision_score(y_valid, p_valid))
print('ROC-AUC:', roc_auc_score(y_valid, p_valid))


In [ ]:

prices_ = prices.copy()
prices_['price_eur'] = np.where(prices_['currency']=='EUR', prices_['price_per_uom'], prices_['price_per_uom']/1.09)
results = []
for (sid, sku), g in prices_.groupby(['supplier_id','sku']):
    g = g.sort_values('valid_from').copy()
    x = np.log1p(g['price_eur'])
    med = np.median(x)
    mad = np.median(np.abs(x - med)) or 1e-6
    z = 0.6745*(x - med)/mad
    g['robust_z'] = z
    top = g.loc[g['robust_z'].abs().sort_values(ascending=False).head(3).index]
    for _, r in top.iterrows():
        results.append({'supplier_id': sid, 'sku': sku, 'valid_from': r['valid_from'], 'price_eur': r['price_eur'], 'robust_z': r['robust_z']})
import pandas as pd
pd.DataFrame(results).sort_values('robust_z', key=lambda s: s.abs(), ascending=False).head(10)
